In [ ]:
def load_extended_acl(file_path):
    """Reads extended ACL rules from a file."""
    acl_rules = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if parts[0] == "access-list":
                rule = {
                    "id": parts[1],
                    "action": parts[2],  
                    "protocol": parts[3],  
                    "source_ip": parts[4],
                    "source_mask": parts[5],
                    "dest_ip": parts[6],
                    "dest_mask": parts[7],
                    "port": parts[8] if len(parts) > 8 else None
                }
                acl_rules.append(rule)
    return acl_rules

def process_extended_acl(acl_rules, packet_file):
    """Processes incoming packets for extended ACL."""
    with open(packet_file, 'r') as file:
        for line in file:
            parts = line.strip().split()
            src_ip, dest_ip, port = parts[0], parts[1], parts[2]
            action = "denied"
            for rule in acl_rules:
                if matches_extended_acl(src_ip, dest_ip, port, rule):
                    action = rule["action"]
                    break
            print(f"Packet from {src_ip} to {dest_ip} on port {port} {action}")

def matches_extended_acl(src_ip, dest_ip, port, rule):
    """Checks if an IP and port match an ACL rule."""
    import ipaddress
    src_net = ipaddress.IPv4Network(f"{rule['source_ip']}/{rule['source_mask']}", strict=False)
    dest_net = ipaddress.IPv4Network(f"{rule['dest_ip']}/{rule['dest_mask']}", strict=False)
    if ipaddress.IPv4Address(src_ip) in src_net and ipaddress.IPv4Address(dest_ip) in dest_net:
        if rule["port"] is None or rule["port"] == port:
            return True
    return False


acl_file = "extended_acl2.txt"
packet_file = "extended_packets2.txt"


acl_rules = load_extended_acl(acl_file)
process_extended_acl(acl_rules, packet_file)


Packet from 10.10.1.20 to 10.10.5.5 on port 53 deny
Packet from 10.10.2.50 to 10.10.5.100 on port 22 permit
Packet from 10.10.3.100 to 10.10.6.5 on port 53 permit
Packet from 192.168.1.1 to 10.10.5.5 on port 80 denied
Packet from 10.10.4.10 to 10.10.5.50 on port 443 permit
